# 导入库及自定义函数

## 库

In [1]:
import cv2
import numpy as np
import sys
from matplotlib import pyplot as plt
import os, random, shutil
import torch
import torch.nn.functional as F
import torch.optim as optim
import torch.nn as nn
from itertools import cycle
from torchvision import datasets
from torch.autograd import Variable
import torchvision
from torchvision.transforms import transforms
from visdom import Visdom
from torch.utils.data import Dataset, DataLoader, TensorDataset,ConcatDataset

## 自定义函数

In [2]:
def pltshow(img):
    """
    介绍：在Jupyter界面上展示图片。转变了numpy数组的颜色空间，使图片色彩符合直觉 \n
    传入变量： \n
            img:传入的图片 \n
    传出变量： \n
            无 \n
    """
    
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) #颜色空间转换函数
    plt.imshow(img, cmap = 'gray', interpolation = 'bicubic')
    plt.xticks([]), plt.yticks([]) # 隐藏 X 和 Y 轴的刻度值
    plt.show()

# 训练集制作

## 标签定义
    健康-0
    caries-1
    Calculus-2
    Gingivitis-3
    hypodontia-4
    Discoloration-5
    Ulcer-6

## 移动

In [32]:
"""
type_name="Ulcer"
dir_name="tooth/Ulcer/preview/"

save_dir="datas/"
name_list=os.listdir(dir_name)
#print(name_list)
sum=-1
for name in name_list:
    sum+=1
    with open('datas/label/'+type_name+str(sum)+'.txt','w') as f1:
        f1.write('6')
    im=cv2.imread(dir_name+name)
    cv2.imwrite('datas/images/'+type_name+str(sum)+'.jpg',im)
"""
print("完成")

完成


In [4]:
"""
img_dir='images/'
name_list=os.listdir(img_dir)
sum=0
for name in name_list:
    if os.path.exists(img_dir+name+'/Out_image.jpg'):
        sum+=1
        with open(img_dir+name+'/label.txt') as f1:
            with open('datas/label/'+str(sum)+'.txt','w') as f2:
                f2.write(f1.read())
        im=cv2.imread(img_dir+name+'/Ori_image.jpg')
        cv2.imwrite('datas/images/'+str(sum)+'.jpg',im)
"""

"\nimg_dir='images/'\nname_list=os.listdir(img_dir)\nsum=0\nfor name in name_list:\n    if os.path.exists(img_dir+name+'/Out_image.jpg'):\n        sum+=1\n        with open(img_dir+name+'/label.txt') as f1:\n            with open('datas/label/'+str(sum)+'.txt','w') as f2:\n                f2.write(f1.read())\n        im=cv2.imread(img_dir+name+'/Ori_image.jpg')\n        cv2.imwrite('datas/images/'+str(sum)+'.jpg',im)\n"

## 划分训练集和验证集

In [31]:
"""
def moveimg(fileDir, tarDir):
    pathDir = os.listdir(fileDir)  # 取图片的原始路径
    filenumber = len(pathDir)
    rate = 0.1  # 自定义抽取图片的比例，比方说100张抽10张，那就是0.1
    picknumber = int(filenumber * rate)  # 按照rate比例从文件夹中取一定数量图片
    sample = random.sample(pathDir, picknumber)  # 随机选取picknumber数量的样本图片
    print(sample)
    for name in sample:
        shutil.move(fileDir + name, tarDir + "/" + name)
    return
 
def movelabel(file_list, file_label_train, file_label_val):
    for i in file_list:
        if i.endswith('.jpg'):
            # filename = file_label_train + "\\" + i[:-4] + '.xml'  # 可以改成xml文件将’.txt‘改成'.xml'就可以了
            filename = file_label_train + "/" + i[:-4] + '.txt'  # 可以改成xml文件将’.txt‘改成'.xml'就可以了
            if os.path.exists(filename):
                shutil.move(filename, file_label_val)
                print(i + "处理成功！")
if __name__ == '__main__':
    fileDir = r'datas/images'+ "/"  # 源图片文件夹路径
    tarDir = r'datas/images_val'  # 图片移动到新的文件夹路径
    moveimg(fileDir, tarDir)
    file_list = os.listdir(tarDir)
    file_label_train = 'datas/label' # 源图片标签路径
    file_label_val = 'datas/label_val'  # 标签
      # 移动到新的文件路径
    movelabel(file_list, file_label_train, file_label_val)
"""

['Discoloration725.jpg', 'caries1555.jpg', 'Ulcer2403.jpg', 'Discoloration236.jpg', 'caries2257.jpg', 'Gingivitis1362.jpg', 'Discoloration468.jpg', 'Discoloration1640.jpg', 'Ulcer971.jpg', 'Ulcer158.jpg', 'Discoloration1085.jpg', 'Ulcer432.jpg', 'Gingivitis1627.jpg', 'Calculus32.jpg', 'caries2046.jpg', 'Ulcer449.jpg', 'Discoloration561.jpg', 'Discoloration10.jpg', 'Ulcer1643.jpg', 'Discoloration803.jpg', 'Ulcer418.jpg', 'caries2318.jpg', 'Calculus1149.jpg', 'Calculus854.jpg', 'caries2294.jpg', 'Gingivitis304.jpg', 'Ulcer1098.jpg', 'hypodontia622.jpg', 'caries464.jpg', 'Ulcer2281.jpg', 'Ulcer23.jpg', 'Discoloration571.jpg', 'caries2095.jpg', 'Gingivitis1195.jpg', 'Discoloration1057.jpg', 'Ulcer869.jpg', 'Ulcer2414.jpg', 'Calculus120.jpg', 'Ulcer1027.jpg', 'Ulcer727.jpg', 'Gingivitis1973.jpg', 'hypodontia770.jpg', 'caries880.jpg', 'Gingivitis455.jpg', 'Ulcer452.jpg', 'Gingivitis1341.jpg', 'Calculus108.jpg', 'Gingivitis1308.jpg', 'Gingivitis627.jpg', 'caries2205.jpg', 'Gingivitis1365.jpg'

# 训练

## 相关自定义函数

### DataLoader制作函数

In [33]:
def Make_loader(data_train,data_label,batch_size=200, shuffle=False):
    """
    介绍：制作DataLoader \n
    传入变量: \n
            data_train:传入的图片集合 \n
            data_label:传入的标签集合 \n
            batch_size:每一组的数据数量，用于划分训练集 \n
            shuffle:是否对传入的数据及进行随机打乱 \n
    传出变量： \n
            data_loader:传出制作好的DataLoader \n
    """
    
    dataset = TensorDataset(data_train,data_label)
    data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)#shuffle是是否打乱数据集，可自行设置

    return data_loader

### 数据集读取和制作函数

In [34]:
def get_loader(fileDir,tarDir,file_label_train,file_label_val,batch_size=200,shuffle=False):
    """
    介绍：从文件中读取数据并制作数据集 \n
    传入变量: \n
            fileDir:训练图片文件夹路径 \n
            tarDir:验证图片文件夹路径 \n
            file_label_train:训练标签文件夹路径 \n
            file_label_val:验证标签文件夹路径 \n
            batch_size:每一组的数据数量，用于划分训练集 \n
            shuffle:是否对传入的数据及进行随机打乱 \n
    传出变量： \n
            train_loader:传出制作好的训练数据集 \n
            test_loader:传出制作好的验证数据集 \n
    """
    
    transf = torchvision.transforms.ToTensor()  # 实例化类
    device=torch.device('cuda:0')  #在GPU上运行
    
    img_list=[]
    label_list=[]
    img_list_val=[]
    label_list_val=[]
    name_list = os.listdir(fileDir)
    name_val_list = os.listdir(tarDir)
    for name in name_list:
        f=open(file_label_train+'/'+name[:-4]+'.txt')
        text=f.read()
        if text=='':
            print('训练集 '+name+' 的标签为空')
            f.close()
        else:
            im=cv2.imread(fileDir+'/'+name)
            #lenth=max([im.shape[0],im.shape[1]])
            label_list.append(int(text))
            im=cv2.resize(im,(128,128), interpolation = cv2.INTER_CUBIC)
            img_list.append(transf(im))
            f.close

    for name in name_val_list:
        f=open(file_label_val+'/'+name[:-4]+'.txt')
        text=f.read()
        if text=='':
            print('训练集 '+name+' 的标签为空')
            f.close()
        else:
            im=cv2.imread(tarDir+'/'+name)
            #lenth=max([im.shape[0],im.shape[1]])
            label_list_val.append(int(text))
            im=cv2.resize(im,(128,128), interpolation = cv2.INTER_CUBIC)
            img_list_val.append(transf(im))
            f.close


    data=torch.stack(img_list)
    data_val=torch.stack(img_list_val)
    target=torch.tensor(label_list)
    target_val=torch.tensor(label_list_val)

    data=data.to(torch.float32)
    data_val=data_val.to(torch.float32)
    target=target.to(torch.long)
    target_val=target_val.to(torch.long)
                        
    print('')
    print('训练图片集规模：',data.shape)
    print('训练标签集规模：',target.shape)
    print('验证图片集规模：',data_val.shape)
    print('验证标签集规模：',target_val.shape)
    
    train_loader,test_loader=Make_loader(data,target,batch_size,shuffle) , Make_loader(data_val,target_val,batch_size,False)
    return train_loader,test_loader  

### ResNet34

In [45]:
class CommonBlock(nn.Module):
    def __init__(self, in_channel, out_channel, stride):
        super(CommonBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channel, out_channel, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channel)
        self.conv2 = nn.Conv2d(out_channel, out_channel, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channel)

    def forward(self, x):
        identity = x

        x = F.relu(self.bn1(self.conv1(x)), inplace=True)
        x = self.bn2(self.conv2(x))

        x += identity
        return F.relu(x, inplace=True)

class SpecialBlock(nn.Module):
    def __init__(self, in_channel, out_channel, stride):
        super(SpecialBlock, self).__init__()
        self.change_channel = nn.Sequential(
            nn.Conv2d(in_channel, out_channel, kernel_size=1, stride=stride[0], padding=0, bias=False),
            nn.BatchNorm2d(out_channel)
        )
        self.conv1 = nn.Conv2d(in_channel, out_channel, kernel_size=3, stride=stride[0], padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channel)
        self.conv2 = nn.Conv2d(out_channel, out_channel, kernel_size=3, stride=stride[1], padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channel)

    def forward(self, x):
        identity = self.change_channel(x)

        x = F.relu(self.bn1(self.conv1(x)), inplace=True)
        x = self.bn2(self.conv2(x))

        x += identity
        return F.relu(x, inplace=True)


class ResNet34(nn.Module):
    def __init__(self):
        super(ResNet34, self).__init__()
        self.prepare = nn.Sequential(
            nn.Conv2d(3, 64, 7, 2, 3),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(3, 2, 1)
        )
        self.layer1 = nn.Sequential(
            CommonBlock(64, 64, 1),
            CommonBlock(64, 64, 1),
            CommonBlock(64, 64, 1)
        )
        self.layer2 = nn.Sequential(
            SpecialBlock(64, 128, [2, 1]),
            CommonBlock(128, 128, 1),
            CommonBlock(128, 128, 1),
            CommonBlock(128, 128, 1)
        )
        self.layer3 = nn.Sequential(
            SpecialBlock(128, 256, [2, 1]),
            CommonBlock(256, 256, 1),
            CommonBlock(256, 256, 1),
            CommonBlock(256, 256, 1),
            CommonBlock(256, 256, 1),
            CommonBlock(256, 256, 1)
        )
        self.layer4 = nn.Sequential(
            SpecialBlock(256, 512, [2, 1]),
            CommonBlock(512, 512, 1),
            CommonBlock(512, 512, 1)
        )
        self.pool = nn.AdaptiveAvgPool2d(output_size=(1, 1))
        self.fc = nn.Sequential(
            nn.Dropout(p=0.5),
            nn.Linear(512, 256),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5),
            nn.Linear(256, 7)  #分类数在这改
        )

    def forward(self, x):
        x = self.prepare(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.pool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fc(x)

        return x

### 网络训练函数

In [ ]:
def train(model,train_loader,test_loader,learn_rate,epochs,batch_size,save_dir):
    """
    介绍：网路训练函数。训练可中断，同时保留训练效果。本函数使用了visdom以可视化数据。使用命令 python -m visdom.server 打开visdom \n
         可视化训练效果网址：http://localhost:8097 \n
    传入变量: \n
            model:要训练的模型 \n
            train_loader:训练数据集 \n
            test_loader:验证数据集 \n
            tarDir:验证图片文件夹路径 \n
            learn_rate:学习率 \n
            epochs:训练轮数 \n
            batch_size:每一组的数据数量，用于划分训练集 \n
            save_dir:存储模型的文件夹 \n
    传出变量： \n
            无 \n
    """
    
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    #device=torch.device('cuda:0')  #在GPU上运行

    optimizer=torch.optim.Adam(model.parameters(), learn_rate)
    loss_fun=nn.CrossEntropyLoss().to(device)
    
    if save_dir[-1]!="/":
        save_dir=save_dir+'/'

    """
    #使用命令 python -m visdom.server 打开visdom
    #网址：http://localhost:8097
    viz=Visdom()
    viz.line([0.],[0.],win='train_loss',opts=dict(title='train loss'))
    viz.line([[0.0,0.0]],[0.],win='test',opts=dict(title='test loss&acc',
                                               legend=['loss','acc']))
    """

    best=0
    step_Tr=0
    step_Te=0

    for epoch in range(epochs):
    
        for batch_idx,(data,target) in enumerate(train_loader):
        
            step_Tr+=1
        
            data,target=data.to(device),target.to(device)
        
            logits=model(data)
            loss=loss_fun(logits,target)
        
            #viz.line([loss.item()],[step_Tr],win='train_loss',update='append')
        
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        
            if batch_idx %100.==0:
                print('训练轮数: {} [{}/{} ({:.0f}%) ]\tLoss: {:.9f}'.format(
                    epoch,batch_idx*len(data),len(train_loader.dataset),
                        100*(batch_idx*len(data))/len(train_loader.dataset),loss.item()))
        test_loss=0
        correct=0
        step_Te+=1
        
        for data,target in test_loader:
        
            data,target=data.to(device),target.to(device)
            
            logits=model(data)
            test_loss+=loss_fun(logits,target).item()*len(data)
    
            pred=logits.data.max(1)[1]
            correct +=pred.eq(target.data).sum()
                    
        test_loss/=len(test_loader.dataset)

        #viz.line([[test_loss,correct.cpu()/len(test_loader.dataset)]],
                        #[step_Te],win='test',update='append')
    
        print('\n结果验证: 平均损失: {:.9f},准确率: {}/{} ({:.0f}%)\n'.format(
            test_loss,correct,len(test_loader.dataset), 
        100.*correct/len(test_loader.dataset)))
        
        if 100.*correct/len(test_loader.dataset)>best:
            best=100.*correct/len(test_loader.dataset)
            model_save(model,save_dir+str(batch_size)+"|"+str(epoch)+'|'+str(float(100.*correct/len(test_loader.dataset)))+'.pth')
            print("<--------->")
            print("第"+str(epoch)+"轮保存成功")
            print("<--------->")

    print('训练结束！')

### 网络保存函数

In [47]:
def model_save(model,path):
    """
    介绍：保存模型为权重文件 \n
    传入变量: \n
            model:要保存权重的模型 \n
            path:要保存的路径 \n
    传出变量： \n
            无 \n
    """
    torch.save(model.state_dict(),path)

### 权重加载函数

In [48]:
def model_load(model,path):
    """
    介绍：将权重文件加载到模型上 \n
    传入变量: \n
            model:要加载权重的模型 \n
            path:要加载的权重的路径 \n
    传出变量： \n
            无 \n
    """
    model.load_state_dict(torch.load(path))

### 合并两个DataLoader

In [49]:
def Merge_loader(Loader_1, Loader_2, batch_size, shuffle):
    """
    介绍：本函数可混合两个DataLoader \n
    传入变量: \n
            Loader_1:混合对象1 \n
            Loader_2:混合对象2 \n
            batch_size:每个batch的size \n
            shuffle:是否随即打乱数据 \n
    传出变量： \n
            merged_dataloader:混合后的DataLoader \n
    """
    
    # 将两个 DataLoader 的数据集合并为一个 ConcatDataset 对象
    dataset_1=Loader_1.dataset
    
    img_list=[]
    label_list=[]
    for x in range(len(Loader_2.dataset)):
        img_list.append(Loader_2.dataset[x][0])
        label_list.append(Loader_2.dataset[x][1])
        
    data=torch.stack(img_list)
    target=torch.tensor(label_list)
    data=data.to(torch.float32)
    target=target.to(torch.long)
    
    dataset_2 = TensorDataset(data,target)
        
    concat_dataset = ConcatDataset([dataset_1,dataset_2])
    
    # 创建合并后的 DataLoader 对象
    merged_dataloader = DataLoader(concat_dataset, batch_size=batch_size, shuffle=shuffle)

    return merged_dataloader

## 网络训练

### 需要的参数

In [ ]:
learn_rate=1e-4   #学习率，防止过拟合
epochs=800   #训练轮数
batch_size=10   #数据集的划分量
shuffle=True   #生成数据集时是否打乱

### 制作data_loader

In [51]:
train_loader,test_loader=get_loader('datas/images','datas/images_val','datas/label','datas/label_val',batch_size,shuffle)


训练图片集规模： torch.Size([10555, 3, 128, 128])
训练标签集规模： torch.Size([10555])
验证图片集规模： torch.Size([1172, 3, 128, 128])
验证标签集规模： torch.Size([1172])


### 新建网络

In [52]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
let=ResNet34().to(device)

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


### 训练(可中断，模型传入为地址)

In [43]:
"""
本函数使用了visdom以可视化数据
使用命令 python -m visdom.server 打开visdom
可视化训练效果网址：http://localhost:8097
"""
train(let,train_loader,test_loader,learn_rate,epochs,batch_size,"models")

Setting up a new session...
../aten/src/ATen/native/cuda/Loss.cu:240: nll_loss_forward_reduce_cuda_kernel_2d: block: [0,0,0], thread: [0,0,0] Assertion `t >= 0 && t < n_classes` failed.
../aten/src/ATen/native/cuda/Loss.cu:240: nll_loss_forward_reduce_cuda_kernel_2d: block: [0,0,0], thread: [1,0,0] Assertion `t >= 0 && t < n_classes` failed.
../aten/src/ATen/native/cuda/Loss.cu:240: nll_loss_forward_reduce_cuda_kernel_2d: block: [0,0,0], thread: [2,0,0] Assertion `t >= 0 && t < n_classes` failed.
../aten/src/ATen/native/cuda/Loss.cu:240: nll_loss_forward_reduce_cuda_kernel_2d: block: [0,0,0], thread: [3,0,0] Assertion `t >= 0 && t < n_classes` failed.
../aten/src/ATen/native/cuda/Loss.cu:240: nll_loss_forward_reduce_cuda_kernel_2d: block: [0,0,0], thread: [4,0,0] Assertion `t >= 0 && t < n_classes` failed.
../aten/src/ATen/native/cuda/Loss.cu:240: nll_loss_forward_reduce_cuda_kernel_2d: block: [0,0,0], thread: [5,0,0] Assertion `t >= 0 && t < n_classes` failed.
../aten/src/ATen/native/

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
